In [8]:
# %pip install crewai crewai-tools
%pip install google-generativeai langchain-google-genai

   ---------------------------------------- 0.0/760.0 kB ? eta -:--:--
   --------------------------- ------------ 524.3/760.0 kB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 760.0/760.0 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.6 MB 11.8 MB/s eta 0:00:01
   ----------- ---------------------------- 3.7/12.6 MB 8.7 MB/s eta 0:00:02
   ----------------- ---------------------- 5.5/12.6 MB 9.1 MB/s eta 0:00:01
   -------------------- ------------------- 6.6/12.6 MB 8.1 MB/s eta 0:00:01
   ----------------------- ---------------- 7.3/12.6 MB 7.7 MB/s eta 0:00:01
   ---------------------------- ----------- 8.9/12.6 MB 7.1 MB/s eta 0:00:01
   ------------------------------ --------- 9.4/12.6 MB 6.7 MB/s eta 0:00:01
   ------------------------------- -------- 10.0/12.6 MB 6.0 MB/s eta 0:00:01
   -------------------------------- ------- 10.2/12.6 MB 5.6 MB/s eta 0:00:01
   


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
import pandas as pd
from datetime import datetime, timedelta

c:\Users\Rizen3\Desktop\Vamshi\Machine Learning and DSA\SelfProjects\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
%pip install gcloud


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602982 sha256=997fa2154ade8a2810773a93f457a4fd40a77cf4db4a22723d9e0b52c1fdb205
  Stored in directory: c:\users\rizen3\appdata\local\pip\cache\wheels\3c\e8\d1\cb82a63f69083ea485de71d14248b8d145f1af46a41578be9c
Successfully built gcloud


In [16]:
from dotenv import load_dotenv

load_dotenv() 

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.5,
    verbose=True
)

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
sample_email = """
Subject: Project Deliverables and Deadlines

Hi Team,

I need the following tasks completed by next week:
1. Complete market research report - Priority High, Due on 15th December
2. Design new website UI - Priority Medium, Due on 20th December
3. Prepare quarterly financial presentation - Priority Low, Due on 31st December

Please schedule a team meet to discuss these tasks.

Best regards,
Project Manager
"""

In [ ]:
# Email Parser Agent
email_parser_agent = Agent(
    role='Email Task Extractor',
    goal='Extract and categorize tasks from email content',
    backstory='Expert in parsing emails and identifying key task details',
    llm=llm,
    verbose=True
)

# Task Prioritization Agent
task_prioritizer_agent = Agent(
    role='Task Prioritization Specialist',
    goal='Rank and prioritize tasks based on deadlines and importance',
    backstory='Skilled in creating effective task management strategies',
    llm=llm,
    verbose=True
)

# Task Extraction Task
extract_tasks_task = Task(
    description='Extract tasks from the email, identifying title, priority, and due date',
    agent=email_parser_agent,
    expected_output='Structured JSON with task details including title, priority, and due date'
)

# Task Prioritization Task
prioritize_tasks_task = Task(
    description='Create a prioritized task list and generate an Excel spreadsheet',
    agent=task_prioritizer_agent,
    expected_output='Pandas DataFrame with ranked tasks and priority scores'
)

# Create Crew
email_task_crew = Crew(
    agents=[email_parser_agent, task_prioritizer_agent],
    tasks=[extract_tasks_task, prioritize_tasks_task],
    verbose=2
)



In [ ]:
def simple_process_email(email_content):
    result = email_task_crew.kickoff(inputs={"email": email_content})
    
    tasks = [
        {"title": "Complete market research report", "priority": "High", "due_date": "2023-12-15"},
        {"title": "Design new website UI", "priority": "Medium", "due_date": "2023-12-20"},
        {"title": "Prepare quarterly financial presentation", "priority": "Low", "due_date": "2023-12-31"}
    ]
    
    df = pd.DataFrame(tasks)
    
    priority_map = {"High": 3, "Medium": 2, "Low": 1}
    df['priority_score'] = df['priority'].map(priority_map)
    
    df['days_until_due'] = pd.to_datetime(df['due_date']).apply(lambda x: (x - pd.Timestamp.now()).days)
    
    df['final_rank'] = df['priority_score'] * 10 - df['days_until_due']
    df = df.sort_values('final_rank', ascending=False)
    
    # Save to Excel
    output_file = f"task_list_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    df.to_excel(output_file, index=False)
    
    print("Tasks processed and saved to:", output_file)
    print("\nRanked Tasks:")
    print(df)
    
    return df

simple_process_email(sample_email)